In [1]:
import json
import pandas as pd

In [2]:
class Fault(object):
    def __init__(self, id, ip, iter, bit, func, latency):
        self.id = id
        self.ip = ip
        self.iter = iter
        self.bit = bit
        self.func = func
        self.latency = latency
        
    
    
    def __repr__(self):
        return str({'ID': self.id, 'IP': self.ip, 'iter': self.iter, 'bit': self.bit, 'latency': self.latency})
        print("(", self.id, self.ip, self.iter, self.bit, self.latency, ")")
        # print("(ID: %s, IP: %s, Iter: %s, Bit: %d, Latency: %d)" % (self.id, self.ip, self.iter, self.bit, self.latency))

    def __str__(self):
        return str({'ID': self.id, 'IP': self.ip, 'iter': self.iter, 'bit': self.bit, 'latency': self.latency})
    
    def __lt__(self, other):
        return self.id < other.id
        
def json_parser(ffile, filter='SIGSEGV'):
    ID = list()
    IP = list()
    IT = list()
    BIT = list()
    Func = list()
    Latency = list()
    
    with open(ffile) as fh:
        for line in fh.readlines():
            record = json.loads(line)

            status = record['status']
            if status != filter:
                continue
            
            ID.append(record['id'])
            IP.append(record['fault']['dyn_inst'][0])
            IT.append(record['fault']['dyn_inst'][1])
            BIT.append(record['fault']['bit_flipped'])
            Func.append(record['fault']['parent_func'])
            Latency.append(len(record['track']))
    d = {'ID': ID, 'IP': IP, 'Iter': IT, 'Bit': BIT, 'Func': Func, 'Latency': Latency}
    return pd.DataFrame(d)

In [4]:
ffile = '/home/cchen/Documents/Projects/CARE/Benchmarks/lulesh/O1/screen/screen_faults.json'
data = json_parser(ffile)
print(len(data))

6989


In [6]:
ranges = [0, 10, 50, 100, 400, 450]
latency = data['Latency']
dist = latency.groupby(pd.cut(latency, ranges)).count()
print(dist)

Latency
(0, 10]       6049
(10, 50]       529
(50, 100]       65
(100, 400]      13
(400, 450]     333
Name: Latency, dtype: int64


In [8]:
funcs = data['Func']
print(funcs.value_counts())

std::vector<double, std::allocator<double> >::operator[]    1378
CalcEnergyForElems                                          1013
CalcPressureForElems                                         827
CalcElemFBHourglassForce                                     674
EvalEOSForElems                                              569
CalcFBHourglassForceForElems                                 350
CollectDomainNodesToElemNodes                                242
CalcElemNodeNormals                                          161
IntegrateStressForElems                                      144
CalcKinematicsForElems                                       138
CalcMonotonicQGradientsForElems                              104
CalcHourglassControlForElems                                 102
CalcElemVolumeDerivative                                      84
Domain::y                                                     83
Domain::zd                                                    82
Domain::yd               